In [1]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'H2O'
geometry = None
basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='JW')
### HAMILTONIAN end

#####################################

Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(QubitHamiltonian)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
anti_commuting_sets

Building Graph Edges: 100%|##########| 1390/1390 [01:23<00:00, 16.65it/s] 


{0: [(-46.46560078368172+0j) []],
 1: [(0.22003977334376118+0j) [Z8 Z9]],
 2: [(1.3692852793098762+0j) [Z8],
  (0.006509359453068306+0j) [Y0 X1 X8 Y9],
  (0.008126859402904007+0j) [X0 Z1 Z2 X3 Y8 Y9],
  (0.008126859402904007+0j) [Y1 X2 X8 Y9],
  (0.003347900046671431+0j) [X0 Z1 Z2 Z3 Z4 Z5 Z6 X7 Y8 Y9],
  (0.003347900046671431+0j) [Y1 Z2 Z3 Z4 Z5 X6 X8 Y9],
  (0.003880148813422984+0j) [Y1 Z2 Z3 Z4 Z5 Z6 Z7 Y8 X9 X10],
  (-0.003880148813422984+0j) [X0 Z1 Z2 Z3 Z4 Z5 Z6 Z7 X8 X9 Z10 X11]],
 3: [(1.3692852793098764+0j) [Z9],
  (0.006509359453068306+0j) [X0 Y1 Y8 X9],
  (0.008126859402904007+0j) [X1 Y2 Y8 X9],
  (0.008126859402904007+0j) [Y0 Z1 Z2 Y3 X8 X9],
  (0.003347900046671431+0j) [Y0 Z1 Z2 Z3 Z4 Z5 Z6 Y7 X8 X9],
  (0.003347900046671431+0j) [X1 Z2 Z3 Z4 Z5 Y6 Y8 X9],
  (-0.003880148813422984+0j) [Y0 Z1 Z2 Z3 Z4 Z5 Z6 Z7 Y8 Y9 Z10 Y11],
  (0.003880148813422984+0j) [X1 Z2 Z3 Z4 Z5 Z6 Z7 X8 Y9 Y10]],
 4: [(12.412562749529512+0j) [Z1],
  (-0.12510154802574205+0j) [X1 Z2 X3],
  (-0.0426047

In [8]:
from quchem.Ansatz_Generator_Functions import *

##
NOON_spins_combined, NMO_basis = Hamilt.Get_NOON()
##
Hamilt.Get_CCSD_Amplitudes()



ansatz_obj = Ansatz(Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)
reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
    NOON=NOON_spins_combined,
    occ_threshold= 1.997,
    unocc_threshold=1e-4,
    indices_to_remove_list_manual=None, 
    single_cc_amplitudes=Hamilt.molecule.single_cc_amplitudes,
    double_cc_amplitudes=Hamilt.molecule.double_cc_amplitudes,
    singles_hamiltonian=Hamilt.singles_hamiltonian,
    doubles_hamiltonian=Hamilt.doubles_hamiltonian,
    tol_filter_small_terms=None)
reduced_Sec_Quant_CC_ops_ijab

[-1.0 [6^ 8^ 10 12] +
 1.0 [12^ 10^ 8 6],
 -1.0 [7^ 9^ 11 13] +
 1.0 [13^ 11^ 9 7],
 -1.0 [6^ 7^ 10 11] +
 1.0 [11^ 10^ 7 6],
 -1.0 [6^ 7^ 10 13] +
 1.0 [13^ 10^ 7 6],
 -1.0 [6^ 7^ 12 13] +
 1.0 [13^ 12^ 7 6],
 -1.0 [6^ 9^ 10 11] +
 1.0 [11^ 10^ 9 6],
 -1.0 [6^ 9^ 10 13] +
 1.0 [13^ 10^ 9 6],
 -1.0 [6^ 9^ 12 13] +
 1.0 [13^ 12^ 9 6],
 -1.0 [8^ 9^ 10 11] +
 1.0 [11^ 10^ 9 8],
 -1.0 [8^ 9^ 10 13] +
 1.0 [13^ 10^ 9 8],
 -1.0 [8^ 9^ 12 13] +
 1.0 [13^ 12^ 9 8]]

In [9]:
ia_terms, ijab_terms, ia_theta, ijab_theta = ansatz_obj.Get_ia_and_ijab_terms()

print('REDUCTION')
print('ia_terms', len(ia_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ia))
print('ijab_terms', len(ijab_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ijab))

REDUCTION
ia_terms 20 TO 8
ijab_terms 65 TO 11


In [10]:
NOON_spins_combined

array([1.99999781, 1.99831305, 1.99793565, 1.97573132, 1.97271064,
       0.0278487 , 0.02746282])

In [ ]:
Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(reduced_Sec_Quant_CC_ops_ia,
                                                                                                                    reduced_Sec_Quant_CC_ops_ijab)

full_ansatz_Q_Circ = Ansatz_Circuit(Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab,
             Hamilt.molecule.n_qubits, Hamilt.molecule.n_electrons)

ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(reduced_theta_parameters_ia, reduced_theta_parameters_ijab)

print(ansatz_cirq_circuit.to_text_diagram(transpose=True)) 

In [ ]:

simulator = cirq.Simulator()
result = simulator.compute_amplitudes(ansatz_cirq_circuit, bitstrings=[i for i in range(2 ** len(ansatz_cirq_circuit.all_qubits()))])
result=np.around(result, 5)
print(result.reshape([(2 ** len(ansatz_cirq_circuit.all_qubits())), 1]))

In [ ]:
from quchem.LCU_method import *

In [ ]:
from quchem.Simulating_Quantum_Circuit import *
R_uncorrected, Pn, gamma_l = Get_R_linear_combination(anti_commuting_sets[10], 1)
R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1 = absorb_complex_phases(R_uncorrected)

full_circ = Full_Q_Circuit(Pn, R_corrected_Op_list, R_corr_list, ancilla_amplitudes, Hamilt.molecule.n_qubits, ansatz_cirq_circuit)
output_bin_dict = Get_binary_dict_project(full_circ, Pn, 100000, Hamilt.molecule.n_qubits, ancilla_amplitudes, l1)
expectation_value_by_parity(output_bin_dict) * gamma_l 

In [ ]:
n_shots=10


testing =  VQE_Experiment_LCU_UP(anti_commuting_sets,
                 ansatz_cirq_circuit,
                 n_shots,
                 Hamilt.molecule.n_qubits,
                 N_indices_dict=None)
testing.Calc_Energy()

In [ ]:
Hamilt.molecule.fci_energy

In [ ]:
n_shots=10

def GIVE_ENERGY(theta_ia_theta_jab_list):
    theta_ia = theta_ia_theta_jab_list[:len(reduced_theta_parameters_ia)]
    theta_ijab = theta_ia_theta_jab_list[len(reduced_theta_parameters_ia):]
   
    ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(theta_ia, theta_ijab)

    VQE_exp_LCU =  VQE_Experiment_LCU_UP(anti_commuting_sets,
                     ansatz_cirq_circuit,
                     n_shots,
                     Hamilt.molecule.n_qubits,
                     N_indices_dict=None)#{7:0, 8:1, 9:0, 10:1})
    return VQE_exp_LCU.Calc_Energy().real


In [ ]:
### optimizer
from quchem.Scipy_Optimizer import *
THETA_params=[0 for _ in range(len(reduced_theta_parameters_ia)+len(reduced_theta_parameters_ijab))]

GG = Optimizer(GIVE_ENERGY, THETA_params, 'Nelder-Mead', store_values=True, display_iter_steps=True,
               tol=1e-5,
               display_convergence_message=True)
GG.get_env(50)
GG.plot_convergence()
plt.show()

### Tensorflow

In [ ]:
from quchem.TensorFlow_Opt import *

**gradient is given by**

https://arxiv.org/pdf/1906.08728.pdf

$$\frac{\partial O(\theta)}{\partial \theta}=\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta+\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta+\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle -\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta-\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta-\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle$$

$$\frac{\partial O(\theta)}{\partial \theta} =O(\theta+\pi / 4)-O(\theta-\pi / 4)$$

In [ ]:
def calc_gradient(theta_ia_theta_jab_list):
    
    grad_list=[]
    
    for index, theta in enumerate(theta_ia_theta_jab_list):
        
        new_theta_list = theta_ia_theta_jab_list.copy()
        new_theta_list[index] = theta + np.pi/4
        
        Obs_PLUS = GIVE_ENERGY(new_theta_list)
        
        
        new_theta_list[index] = theta - np.pi/4
        Obs_MINUS = GIVE_ENERGY(new_theta_list)
        
        gradient = Obs_PLUS - Obs_MINUS
        
        grad_list.append((gradient, theta))
        

    return grad_list

note:
this is very SLOW as it has to run a separate experiment TWICE for each parameter before taking a step!

In [ ]:
import random

X0 = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_Sec_Quant_CC_ops_ia) + len(reduced_Sec_Quant_CC_ops_ijab))]

GG = Tensor_Flow_Optimizer(GIVE_ENERGY, X0, 'Adam', calc_gradient, learning_rate=0.1, beta1=0.9,
                            beta2=0.999, store_values=True, display_iter_steps=True)
GG.optimize(50)
GG.plot_convergence()